In [1]:
# Amazon Repatriation Automation
## Jamie needs to concatenate sales data received from markets into one CSV file
#### Concatenate ICLA and Classical Sheet into one CSV
##### Jamie needs to send over Cost Feed from Amazon, Concatenated Sales Feed from markets, ICLA exceptions / Classical Sheet and International Catalogue
### remove all currency symbols from cost data
###### create dashboard from the data

In [2]:
#importing libraries

import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from google.cloud import bigquery


In [3]:
#importing sales data

sales = pd.read_csv('G:\Amazon Repatriation May 2020\Amazon Repat - Amazon Data April.csv', dtype={'ean': str})

In [4]:
#importing cost data

costs = pd.read_csv('G:\Amazon Repatriation May 2020\Amazon Cost Feed Combined - April.csv', dtype={'UPC': np.int64 })

In [5]:
### reduce table size 
costs2 = costs[['Region','UPC','Transfer Price','Dealer Price','Net Unit Price','Currency ']]

In [6]:
### amount of rows in cost data
costs2.shape

(83531, 6)

In [7]:
#keeping any rows with Transfer Price below .30 euros
costs3 = costs2[costs2['Transfer Price'] >= .3] 

In [8]:
### amount of rows in cost data
costs3.shape

(55654, 6)

In [9]:
###check for duplicates rows
sum(costs3.duplicated())

18904

In [10]:
##drop duplicates
costs4 = costs3.drop_duplicates(ignore_index = True)

In [11]:
###check for duplicates rows
sum(costs4.duplicated())

0

In [12]:
#concatenating inventrory owner groups, vendor groups for streamlined data

sales.loc[sales['inventory_owner_group'] == 'UK', 'inventory_owner_group_combined'] = 'UK'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.co.uk', 'inventory_owner_group_combined'] = 'UK'
sales.loc[sales['inventory_owner_group'] == 'DE', 'inventory_owner_group_combined'] = 'DE'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.de', 'inventory_owner_group_combined'] = 'DE'
sales.loc[sales['inventory_owner_group'] == 'IT', 'inventory_owner_group_combined'] = 'IT'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.it', 'inventory_owner_group_combined'] = 'IT'
sales.loc[sales['inventory_owner_group'] == 'FR', 'inventory_owner_group_combined'] = 'FR'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.fr', 'inventory_owner_group_combined'] = 'FR'
sales.loc[sales['inventory_owner_group'] == 'ES', 'inventory_owner_group_combined'] = 'ES'
sales.loc[sales['inventory_owner_group'] == 'www.amazon.es', 'inventory_owner_group_combined'] = 'ES'
sales.loc[sales['inventory_owner_group'] == 'Pan-EU SCO', 'inventory_owner_group_combined'] = 'Pan-EU SCO'
sales.loc[sales['vendor_group'] == 'UMG UK', 'vendor_group2'] = 'UK'
sales.loc[sales['vendor_group'] == 'UMG DE', 'vendor_group2'] = 'DE'
sales.loc[sales['vendor_group'] == 'UMG FR', 'vendor_group2'] = 'FR'
sales.loc[sales['vendor_group'] == 'UMG IT', 'vendor_group2']  ='IT'
sales.loc[sales['vendor_group'] == 'UMG ES', 'vendor_group2'] ='ES'
sales.loc[sales['vendor_group'] == 'UMG AT', 'vendor_group2'] ='AT'
sales.loc[sales['vendor_group'] == '3rd Party', 'vendor_group2'] = '3rd Party'

In [13]:
#creating movement key

sales['movement'] = sales['vendor_group'] + ' to ' +sales['inventory_owner_group_combined'] 

In [14]:
#splitting up strings of UPC Amazon delivers to us

new = sales['ean'].str.split("/", n =-1, expand = True) 
sales["ean1"]= new[0] 
sales["ean2"]= new[1] 
sales["ean3"]= new[2] 
sales["ean4"]= new[3] 
sales["ean5"]= new[4] 
sales.drop(columns =["ean"], inplace = True) 


In [20]:
sales3 = sales.fillna(0)


In [21]:
##change ean columns to int to match up with costs data

sales3['eanone']= sales3['ean1'].astype(float).astype(np.int64)
sales3['eantwo']= sales3['ean2'].astype(float).astype(np.int64)
sales3['eanthree']= sales3['ean3'].astype(float).astype(np.int64)
sales3['eanfour']= sales3['ean4'].astype(float).astype(np.int64)
sales3['eanfive']= sales3['ean5'].astype(float).astype(np.int64)


In [22]:
### drop rows that are redundant 
del sales3['ean1']
del sales3['ean2']
del sales3['ean3']
del sales3['ean4']
del sales3['ean5']

In [24]:
### merge data
combo = sales3.merge(costs4, how='inner', left_on=["eanone"or"eantwo"or"eanthree"or"eanfour"or"eanfive","vendor_group2"], right_on=["UPC","Region"])

In [26]:
###amount of rows
combo.shape

(179979, 23)

In [27]:
### drop row which didn't match
combo4 = combo[combo['UPC'].notna()]

In [28]:
### amount of rows
combo4.shape

(179979, 23)

In [29]:
###check for duplicates rows
sum(combo4.duplicated())

3893

In [30]:
#importing international catalogue data

international_catalogue = pd.read_csv(r'G:\Amazon Repatriation May 2020\UMGI Physical Catalogue_17042020.csv', dtype={'UPC': float})

In [32]:
### drop redundant columns
cat2 = international_catalogue[['Rep Owner','UPC']]

In [34]:
##drop duplicates
cat3 = cat2.drop_duplicates(ignore_index = True)

In [35]:
###check for duplicates rows
sum(cat3.duplicated())

0

In [37]:
##only keep first label
cat4 = cat3.drop_duplicates(subset='UPC', keep="first")

In [39]:
### drop row which didn't match
cat5 = cat4[cat4['UPC'].notna()]

In [40]:
cat6 = cat5['UPC'].astype(float).astype(np.int64)

In [41]:
#merging our dataset and international catalogue to return Rep Owner for each UPC

combo_cat = pd.merge(combo4,cat4[['UPC','Rep Owner']],
                     left_on="eanone"or"eantwo"or"eanthree"or"eanfour"or"eanfive", right_on=['UPC'], how='left')

In [43]:
### count rows and columsn in combo2
combo_cat.shape

(179979, 25)

In [44]:
###drop caroline labels
combo_cat2 = combo_cat[combo_cat['Rep Owner'] != 'Caroline/Astralwerks']

In [45]:
### check row length 
combo_cat2.shape

(179956, 25)

In [46]:
###drop concord labels
combo_cat3 = combo_cat2[combo_cat2['Rep Owner'] != 'Concord']

In [47]:
### check row length
combo_cat3.shape

(179282, 25)

In [49]:
### check row count
combo_cat3.shape

(179282, 25)

In [50]:
#importing exception list

exception_list = pd.read_csv('G:\Amazon Repatriation May 2020\exception_list.csv', dtype={'UPC': np.int64})

In [51]:
#show exception list

exception_list.shape

(6166, 2)

In [52]:
##delete duplicates from exception list

exception_list.drop_duplicates(subset ="UPC", 
                     keep = False, inplace = True)

In [53]:
##show exception list
exception_list.shape

(5677, 2)

In [54]:
##only keep first label
exception_list2 = exception_list.drop_duplicates(subset='UPC', keep="first")

In [55]:
exception_list2.shape

(5677, 2)

In [56]:
#merging data with exception list to find artist exceptions

data_with_exception = pd.merge(combo_cat,exception_list[['UPC','Exception']],left_on="eanone"or"eantwo"or"eanthree"or"eanfour"or"eanfive", right_on=['UPC'], how='left')

In [58]:
##assigning repat percentage, changing Nan to other 50%
data_with_exception['Exception'] = data_with_exception['Exception'].fillna(.50)

In [59]:
##assigning repat percentage, classical 60%, artist exception 40%, other 50%
 
data_with_exception.loc[data_with_exception['Exception'] == .5, 'repat_%'] = .5
data_with_exception.loc[data_with_exception['Exception'] == 'Artist ICLA', 'repat_%'] = .4
data_with_exception.loc[data_with_exception['Exception'] == 'Classical', 'repat_%'] = .6

In [61]:
##change repat % column to float for equations
data_with_exception['repat_%'] = data_with_exception['repat_%'].astype(float)

In [62]:

##Repatriation equation bracket net unit price times repat% price brackets minus transfer price

data_with_exception['Repatriation'] = (data_with_exception['Net Unit Price'] * data_with_exception['repat_%']) - data_with_exception['Transfer Price']

In [64]:
##Repatriation percentage equation  bracket repatiation divded by dealer price bracket times one hundred

data_with_exception['Repatriation_Percentage_%'] = (data_with_exception['Repatriation'] / data_with_exception['Dealer Price']) *100

In [66]:
##total repatriation percentage equation stock movements times repatriation 

data_with_exception['Total_Repatriation'] = data_with_exception['unpacked_units'] * data_with_exception['Repatriation']

In [68]:
#removing any rows with negative Total_Repatriation total 

data_with_exception.drop(data_with_exception.index[data_with_exception['Total_Repatriation'] < 0], inplace = True)

In [69]:
##show data

data_with_exception.shape

(178551, 31)

In [70]:
##remove rows with any NA for Total_Repatriation

data_with_exception = data_with_exception[data_with_exception['Total_Repatriation'].notna()]

In [71]:
data_with_exception.shape

(178551, 31)

In [73]:
data_with_exception.shape

(178551, 31)

In [74]:
##remove rows with any NA for inventory owner group

data_with_exception = data_with_exception[data_with_exception['inventory_owner_group_combined']!= 0]

In [75]:
del data_with_exception['UPC']

In [78]:
data_with_exception.shape

(177842, 30)

In [79]:
## rename columns for google big query upload 
data_with_exception = data_with_exception.rename(columns={'Report Type': 'Report_Type', 'Rep Owner': 'Rep_Owner', 'Transfer Price':'Transfer_Price','Dealer Price':'Dealer_Price','Net Unit Price':'Net_Unit_Price','Currency ':'Currency','repat_%':'repat_percentage','Repatriation_Percentage_%':'Repatriation_Percentage_Final'})

In [80]:
###check for duplicates rows
sum(data_with_exception.duplicated())

3850

In [85]:
data_with_exception.to_gbq('commercial_affairs_analytics.amazon_repatriation_sat_test7_2020', 'umg-uk')

1it [00:43, 43.94s/it]
